In [1]:
import json
from datetime import datetime
import time
import xml.etree.ElementTree as ET
import random
from threading import Thread, Lock, local
from IPython.display import clear_output

In [3]:
#Funzione per trasformare le coordinate nel formato corretto
from pyproj import Transformer

transformer = Transformer.from_crs("epsg:4326", "epsg:32633")

def trans_coord(coord_X,coord_Y):
    x,y = transformer.transform(coord_X, coord_Y)
    return [x,y]

In [4]:
#Generazione orario casuale tra due orari limite
def randomTime(start, end):
    frmt = '%H:%M:%S'

    stime = time.mktime(time.strptime(start, frmt))
    etime = time.mktime(time.strptime(end, frmt))

    ptime = stime + random.random() * (etime - stime)
    dt = datetime.fromtimestamp(time.mktime(time.localtime(ptime)))
    return str(dt.hour).zfill(2)+':'+str(dt.minute).zfill(2)+':'+str(dt.second).zfill(2)

In [5]:
with open('./json_posti_lavoro.json') as json_file:
    posti_lavoro = json.load(json_file)

In [6]:
with open('./POI_scuola.json') as json_file:
    scuole = json.load(json_file)

In [7]:
with open('./POI_ristoro.json') as json_file:
    ristori = json.load(json_file)

In [8]:
with open('./POI_shop_essenziale.json') as json_file:
    shop_essenziali = json.load(json_file)

In [9]:
with open('./POI_turismo.json') as json_file:
    turismo = json.load(json_file)

<h1>Agenti Attivi</h1>

In [10]:
with open('./json_agenti_res.json') as json_file:
    agenti = json.load(json_file)

In [11]:
#Definisco possibili routine
routine_attivi=[[0,7,12],[0,3,4,7,12],[0,1,2,3,4,5,6,7,9,10,12],[0,7,8,10,12],[0,7,8,9,11,12],[0,3,4,5,6,7,8,12]]

In [15]:
#Genero slot temporali per agenti attivi (L'orario generato è di termine dell'attività)
routines={}
for a in agenti:
    r=[]
    r.append(randomTime("06:45:00","09:00:00")) #0 Uscita di casa 
    r.append(randomTime("10:50:00","11:00:00")) #1 Lavoro-Studio
    r.append(randomTime("11:00:01","11:10:00")) #2 Pausa
    r.append(randomTime("12:50:01","13:10:00")) #3 Lavoro-Studio
    r.append(randomTime("14:00:00","14:15:00")) #4 Fine pranzo
    r.append(randomTime("15:50:00","16:00:00")) #5 Lavoro-Studio
    r.append(randomTime("16:00:01","16:10:00")) #6 Pausa
    r.append(randomTime("17:50:00","18:10:00")) #7 Lavoro-Studio
    r.append(randomTime("18:50:00","19:10:00")) #8 Casa-Ape-Spese
    r.append(randomTime("19:50:00","20:10:00")) #9 Casa-Ape-Spese
    r.append(randomTime("21:50:00","22:10:00")) #10 Casa-Rist
    r.append(randomTime("22:50:00","23:10:00")) #11 Casa-Serata
    r.append(randomTime("23:50:00","23:59:59")) #12 Casa
    routines[a['id']]=r

In [16]:
def assegna_attivita(l,plan,a,mov):
    if l==0: #Uscita di casa la mattina
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='home'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=routines[a['id']][l]
        leg=ET.SubElement(plan,'leg')
        leg.attrib['mode']=mov
        
    if l==1 or l==3 or l==5 or l==7: #Lavoro o Studio
        activity=ET.SubElement(plan,'activity')
        if 'posto_lavoro' in a:
            id_lav=a['posto_lavoro']
            p_lavoro=list(filter(lambda pl: pl['id'] == id_lav,posti_lavoro))
            if p_lavoro[0]['essenziale']:
                activity.attrib['type']='work_ess'
                activity.attrib['facility']=str(p_lavoro[0]['id'])
            else:
                activity.attrib['type']='work_noEss'
                activity.attrib['facility']=str(p_lavoro[0]['id'])
            coord=trans_coord(p_lavoro[0]['coord_X'],p_lavoro[0]['coord_Y'])
        else:
            if a['tipo']=='attiva':
                activity.attrib['type']='university'
            else:
                activity.attrib['type']='school'
            id_scuola=a['scuola_appartenenza']
            scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
            coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
            activity.attrib['facility']=str(scuola[0]['id'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=routines[a['id']][l]
        leg=ET.SubElement(plan,'leg')
        leg.attrib['mode']=mov
        
    if l==2 or l==6: #Pausa
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='refreshment'
        if a['tipo'] == 'attiva': 
            if 'posto_lavoro' in a: #Lavoratore
                sez=a['posto_lavoro']
                bar=list(filter(lambda r: r['sezione_censo']<sez+25 and r['sezione_censo']>sez-25 and (r['tipo_edificio']=='bar' or r['tipo_edificio']=='cafe'),ristori))
                trovato=False
                if bar: #Cerco un bar
                    random.shuffle(bar)
                    for b in bar:
                        if b['capienza_oraria'][l]>0:
                            b['capienza_oraria'][l]=b['capienza_oraria'][l]-1
                            coord=trans_coord(b['coord_X'],b['coord_Y'])
                            activity.attrib['facility']=str(b['id'])
                            leg=ET.SubElement(plan,'leg')
                            leg.attrib['mode']='walk'
                            trovato=True
                            break
                if not trovato: #Se non lo trova, pausa in loco
                    id_lav=a['posto_lavoro']
                    p_lavoro=list(filter(lambda pl: pl['id'] == id_lav,posti_lavoro))
                    coord=trans_coord(p_lavoro[0]['coord_X'],p_lavoro[0]['coord_Y'])
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']='walk'
            else: #Studente superiori-università
                sez=a['scuola_appartenenza']
                bar=list(filter(lambda r: r['sezione_censo']<sez+25 and r['sezione_censo']>sez-25 and (r['tipo_edificio']=='bar' or r['tipo_edificio']=='cafe'),ristori))
                trovato=False
                if bar: #Cerco un bar nelle vicinanze della scuola
                    random.shuffle(bar)
                    for b in bar:
                        if b['capienza_oraria'][l]>0:
                            b['capienza_oraria'][l]=b['capienza_oraria'][l]-1
                            coord=trans_coord(b['coord_X'],b['coord_Y'])
                            activity.attrib['facility']=str(b['id'])
                            leg=ET.SubElement(plan,'leg')
                            leg.attrib['mode']='walk'
                            trovato=True
                            break
                if not trovato: #Se non lo trova, pausa in loco
                    id_scuola=a['scuola_appartenenza']
                    scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
                    coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']='walk'
                
        else: #Altrimenti è un bambino, quindi rimane a scuola
            id_scuola=a['scuola_appartenenza']
            scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
            coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']='walk'
            
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=routines[a['id']][l]
        
    if l==4: #Pausa pranzo
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='refreshment'
        if 'posto_lavoro' in a:
            sez=a['posto_lavoro']
        else:
            sez=a['scuola_appartenenza']
            
        if a['tipo']=='attiva': #Suddivido bambini con ragazzi/adulti
            rist=list(filter(lambda r: r['sezione_censo']<sez+25 and r['sezione_censo']>sez-25 ,ristori))
            trovato=False
            if rist: #Cerco un ristorante nelle vicinanze
                random.shuffle(rist)
                for r in rist:
                    if r['capienza_oraria'][l]>0:
                        r['capienza_oraria'][l]=r['capienza_oraria'][l]-1
                        coord=trans_coord(r['coord_X'],r['coord_Y'])
                        activity.attrib['facility']=str(r['id'])
                        leg=ET.SubElement(plan,'leg')
                        leg.attrib['mode']='walk'
                        trovato=True
                        break
            if not trovato: #Se non lo trova, torna a casa a mangiare
                coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                leg=ET.SubElement(plan,'leg')
                leg.attrib['mode']=mov
        else:
            id_scuola=a['scuola_appartenenza']
            scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
            coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
                        
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=routines[a['id']][l]
    
    if l==8 or l==9: #Casa - Aperitivo - Spesa
        def casa():
            activity=ET.SubElement(plan,'activity')
            activity.attrib['type']='home'
            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
        def ape():
            activity=ET.SubElement(plan,'activity')
            activity.attrib['type']='refreshment'
            if 'posto_lavoro' in a: #Lavoratore
                sez=a['posto_lavoro']
                bar=list(filter(lambda r: r['sezione_censo']<sez+10 and r['sezione_censo']>sez-10 and (r['tipo_edificio']=='bar' or r['tipo_edificio']=='pub' or r['tipo_edificio']=='cafe' or r['tipo_edificio']=='ice_cream'),ristori))
                trovato=False
                if bar: #Cerco un bar nelle vicinanze del lavoro
                    random.shuffle(bar)
                    for b in bar:
                        if b['capienza_oraria'][l]>0:
                            b['capienza_oraria'][l]=b['capienza_oraria'][l]-1
                            activity.attrib['facility']=str(b['id'])
                            coord=trans_coord(b['coord_X'],b['coord_Y'])
                            trovato=True
                            break
                if not trovato: #Se non lo trova nelle vicinanze, scelgo un bar a caso
                    bar=list(filter(lambda r: r['tipo_edificio']=='bar' or r['tipo_edificio']=='pub' or r['tipo_edificio']=='cafe' or r['tipo_edificio']=='ice_cream',ristori))
                    trovato=False
                    for b in bar:
                        if b['capienza_oraria'][l]>0:
                            b['capienza_oraria'][l]=b['capienza_oraria'][l]-1
                            activity.attrib['facility']=str(b['id'])
                            coord=trans_coord(b['coord_X'],b['coord_Y'])
                            trovato=True
                            break
                    if not trovato: #Va a casa
                        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
            else:
                if a['tipo'] == 'attiva': #Controllo se ragazzo >15 anni
                    sez=a['scuola_appartenenza']
                    bar=list(filter(lambda r: r['sezione_censo']<sez+10 and r['sezione_censo']>sez-10 and (r['tipo_edificio']=='bar' or r['tipo_edificio']=='pub' or r['tipo_edificio']=='cafe' or r['tipo_edificio']=='ice_cream'),ristori))
                    trovato=False
                    if bar: #Cerco un bar nelle vicinanze 
                        random.shuffle(bar)
                        for b in bar:
                            if b['capienza_oraria'][l]>0:
                                b['capienza_oraria'][l]=b['capienza_oraria'][l]-1
                                activity.attrib['facility']=str(b['id'])
                                coord=trans_coord(b['coord_X'],b['coord_Y'])
                                trovato=True
                                break
                    if not trovato: #Se non lo trova nelle vicinanze, scelgo un bar a caso
                        bar=list(filter(lambda r: r['tipo_edificio']=='bar' or r['tipo_edificio']=='pub' or r['tipo_edificio']=='cafe' or r['tipo_edificio']=='ice_cream',ristori))
                        trovato=False
                        for b in bar:
                            if b['capienza_oraria'][l]>0:
                                b['capienza_oraria'][l]=b['capienza_oraria'][l]-1
                                activity.attrib['facility']=str(b['id'])
                                coord=trans_coord(b['coord_X'],b['coord_Y'])
                                trovato=True
                                break
                        if not trovato: #Va a casa
                            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                else: #Se bambino, aperitivo diventa merenda a casa
                    coord=trans_coord(a['X_residenza'],a['Y_residenza'])

            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
        
        def spesa():
            random.shuffle(shop_essenziali)
            trovato=False
            for se in shop_essenziali:
                if se['capienza_oraria'][l]>0:
                    se['capienza_oraria'][l]=se['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='shop_ess'
                    activity.attrib['facility']=str(se['id'])
                    coord=trans_coord(se['coord_X'],se['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    trovato=True
                    break
            if not trovato: #Eccezionale - Prese le coordinate di casa
                activity=ET.SubElement(plan,'activity')
                activity.attrib['type']='home'
                coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                activity.attrib['x']=str(coord[0])
                activity.attrib['y']=str(coord[1])
                activity.attrib['end_time']=routines[a['id']][l]
                leg=ET.SubElement(plan,'leg')
                leg.attrib['mode']=mov
                
        if a['tipo']=='attiva':
            random.choice([casa,ape,spesa])()
        else:
            random.choice([casa,ape])()
            
    if l==10: #Casa - Ristorante
        def casa():
            activity=ET.SubElement(plan,'activity')
            activity.attrib['type']='home'
            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
        def ristorante():
            random.shuffle(ristori)
            trovato=False
            for r in ristori: #Cerco ristorante
                if r['capienza_oraria'][l]>0 and r['tipo_edificio'] in ['restaurant','fast_food','bbq','pub']:
                    r['capienza_oraria'][l]=r['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='refreshment'
                    activity.attrib['facility']=str(r['id'])
                    coord=trans_coord(r['coord_X'],r['coord_Y'])
                    trovato=True
                    break 
            if not trovato: #Eccezionale - fornisco coord di casa
                activity=ET.SubElement(plan,'activity')
                activity.attrib['type']='home'
                coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
            
        if a['tipo']=='attiva':
            random.choice([casa,ristorante])()
        else:
            casa()
            
    if l==11: #Casa - Serata
        def casa():
            activity=ET.SubElement(plan,'activity')
            activity.attrib['type']='home'
            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
            
        def serata():
            random.shuffle(ristori)
            random.shuffle(turismo)
            if random.choice([True,False]):
                trovato=False
                for r in ristori:
                    if r['capienza_oraria'][l]>0 and r['tipo_edificio'] in ['bar','cafe','ice_cream','pub']:
                        r['capienza_oraria'][l]=r['capienza_oraria'][l]-1
                        activity=ET.SubElement(plan,'activity')
                        activity.attrib['type']='refreshment'
                        activity.attrib['facility']=str(r['id'])
                        coord=trans_coord(r['coord_X'],r['coord_Y'])
                        trovato=True
                        break 
                if not trovato: # Eccezionale - fornite coordinate di casa
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='home'
                    coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                    
                activity.attrib['x']=str(coord[0])
                activity.attrib['y']=str(coord[1])
                activity.attrib['end_time']=routines[a['id']][l]
                leg=ET.SubElement(plan,'leg')
                leg.attrib['mode']=mov
                
            else:
                trovato=False
                for t in turismo:
                    if t['capienza_oraria'][l]>0:
                        t['capienza_oraria'][l]=t['capienza_oraria'][l]-1
                        activity=ET.SubElement(plan,'activity')
                        activity.attrib['type']='night'
                        activity.attrib['facility']=str(t['id'])
                        coord=trans_coord(t['coord_X'],t['coord_Y'])
                        trovato=True
                        break
                if not trovato: # Eccezionale - fornite coordinate di casa
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='home'
                    coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                    
                activity.attrib['x']=str(coord[0])
                activity.attrib['y']=str(coord[1])
                activity.attrib['end_time']=routines[a['id']][l]
                leg=ET.SubElement(plan,'leg')
                leg.attrib['mode']=mov
                        
        if a['tipo']=='attiva':
            random.choice([casa,serata])()
        else:
            casa()
            
    if l==12: #Rientro a casa
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='home'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])

In [17]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

root=ET.Element('population')

for a in agenti:

    person = ET.SubElement(root, 'person')
    person.attrib['id'] = str(a['id'])
        
    if a['automunito']:
        mov='car'
    else:
        mov=random.choice(['walk','bike'])
            
    plan=ET.SubElement(person,'plan')
    
    routine=random.choice(routine_attivi);
    
    for l in routine:
        assegna_attivita(l,plan,a,mov)
        
print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-04-09 13:57:10
Fine = 2021-04-09 14:20:44


________________________

<h1>Agenti non attivi</h1>

In [18]:
#importo json agenti non attivi
with open('./json_agenti_non_attivi_res.json') as json_file:
    agenti_non_attivi = json.load(json_file)

In [19]:
with open('./POI_servizi.json') as json_file:
    servizi = json.load(json_file)

In [20]:
with open('./POI_tempolibero.json') as json_file:
    tempolibero = json.load(json_file)

In [21]:
with open('./POI_sports.json') as json_file:
    sports = json.load(json_file)

In [22]:
with open('./POI_shop_nonessenziale.json') as json_file:
    shop_nonessenziali = json.load(json_file)

In [25]:
#Definisco possibili routine
routine_non_attivi=[[0,1,3,5,7,8,12],[0,1,3,4,7,8,9,11,12],[0,3,5,7,8,10,11,12]]

In [28]:
#Genero slot temporali per agenti non attivi (L'orario generato è di termine dell'attività)
routines={}
for a in agenti_non_attivi:
    r=[]
    r.append(randomTime("06:45:00","09:00:00")) #0 Uscita di casa 
    r.append(randomTime("10:50:00","11:00:00")) #1 Attività 1
    r.append(randomTime("11:00:01","11:10:00")) #2 Non usato
    r.append(randomTime("12:50:01","13:10:00")) #3 Attività 2
    r.append(randomTime("14:00:00","14:15:00")) #4 Casa-Pranzo
    r.append(randomTime("15:50:00","16:00:00")) #5 Casa-Pranzo 
    r.append(randomTime("16:00:01","16:10:00")) #6 Non usato
    r.append(randomTime("17:50:00","18:10:00")) #7 Attività 3
    r.append(randomTime("18:50:00","19:10:00")) #8 Attività 4
    r.append(randomTime("19:50:00","20:10:00")) #9 Casa-Cena
    r.append(randomTime("21:50:00","22:10:00")) #10 Casa-Cena
    r.append(randomTime("22:50:00","23:10:00")) #11 Casa-Serata
    r.append(randomTime("23:50:00","23:59:59")) #12 Casa
    routines[a['id']]=r

In [29]:
def assegna_attivita_non_attivi(l,plan,a,mov):
    
    if l==0: #Uscita di casa la mattina
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='home'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=routines[a['id']][l]
        leg=ET.SubElement(plan,'leg')
        leg.attrib['mode']=mov
        
    if l==1 or l==3 or l==7 or l==8: #Attivita
        def shop_essenziale():
            random.shuffle(shop_essenziali)
            for se in shop_essenziali:
                if se['capienza_oraria'][l]>0:
                    se['capienza_oraria'][l]=se['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='shop_ess'
                    activity.attrib['facility']=str(se['id'])
                    coord=trans_coord(se['coord_X'],se['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        def shop_nonessenziale():
            random.shuffle(shop_nonessenziali)
            for sne in shop_nonessenziali:
                if sne['capienza_oraria'][l]>0:
                    sne['capienza_oraria'][l]=sne['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='shop_noEss'
                    activity.attrib['facility']=str(sne['id'])
                    coord=trans_coord(sne['coord_X'],sne['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        def servizio():
            random.shuffle(servizi)
            for s in servizi:
                if s['capienza_oraria'][l]>0:
                    s['capienza_oraria'][l]=s['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='business'
                    activity.attrib['facility']=str(s['id'])
                    coord=trans_coord(s['coord_X'],s['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        def ristoro():
            random.shuffle(ristori)
            for r in ristori:
                if r['capienza_oraria'][l]>0 and r['tipo_edificio'] in ['bar','cafe','ice_cream','pub']:
                    r['capienza_oraria'][l]=r['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='refreshment'
                    activity.attrib['facility']=str(r['id'])
                    coord=trans_coord(r['coord_X'],r['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        def att_turismo():
            random.shuffle(turismo)
            for t in turismo:
                if t['capienza_oraria'][l]>0:
                    t['capienza_oraria'][l]=t['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='turism'
                    activity.attrib['facility']=str(t['id'])
                    coord=trans_coord(t['coord_X'],t['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        def att_tempolibero():
            random.shuffle(tempolibero)
            for tl in tempolibero:
                if tl['capienza_oraria'][l]>0:
                    tl['capienza_oraria'][l]=tl['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='leisure'
                    activity.attrib['facility']=str(tl['id'])
                    coord=trans_coord(tl['coord_X'],tl['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        def sport():
            random.shuffle(sports)
            for sp in sports:
                if sp['capienza_oraria'][l]>0:
                    sp['capienza_oraria'][l]=sp['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='sport'
                    activity.attrib['facility']=str(sp['id'])
                    coord=trans_coord(sp['coord_X'],sp['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        
        continua=False
        while not continua:
            continua=random.choice([shop_essenziale,shop_nonessenziale,ristoro,servizio,att_turismo,att_tempolibero,sport])()
        
    if l==4 or l==5 or l==9 or l==10: # Casa o Pranzo - Casa o Cena
        def casa():
            activity=ET.SubElement(plan,'activity')
            activity.attrib['type']='home'
            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
            return True
        def ristoro():
            random.shuffle(ristori)
            for r in ristori:
                if r['capienza_oraria'][l]>0 and r['tipo_edificio'] in ['restaurant','fast_food','bbq','pub']:
                    r['capienza_oraria'][l]=r['capienza_oraria'][l]-1
                    activity=ET.SubElement(plan,'activity')
                    activity.attrib['type']='refreshment'
                    activity.attrib['facility']=str(r['id'])
                    coord=trans_coord(r['coord_X'],r['coord_Y'])
                    activity.attrib['x']=str(coord[0])
                    activity.attrib['y']=str(coord[1])
                    activity.attrib['end_time']=routines[a['id']][l]
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']=mov
                    return True
            return False
        
        continua=False
        while not continua:
            continua=random.choice([casa,ristoro])()
        
    if l==11: #Casa o Serata
        def casa():
            activity=ET.SubElement(plan,'activity')
            activity.attrib['type']='home'
            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
            activity.attrib['end_time']=routines[a['id']][l]
            leg=ET.SubElement(plan,'leg')
            leg.attrib['mode']=mov
            return True
        def serata():
            random.shuffle(ristori)
            random.shuffle(turismo)
            if random.choice([True,False]):
                for r in ristori:
                    if r['capienza_oraria'][l]>0 and r['tipo_edificio'] in ['bar','cafe','ice_cream','pub']:
                        r['capienza_oraria'][l]=r['capienza_oraria'][l]-1
                        activity=ET.SubElement(plan,'activity')
                        activity.attrib['type']='refreshment'
                        activity.attrib['facility']=str(r['id'])
                        coord=trans_coord(r['coord_X'],r['coord_Y'])
                        activity.attrib['x']=str(coord[0])
                        activity.attrib['y']=str(coord[1])
                        activity.attrib['end_time']=routines[a['id']][l]
                        leg=ET.SubElement(plan,'leg')
                        leg.attrib['mode']=mov
                        return True 
            else:
                if random.choice([True,False]):
                    for t in turismo:
                        if t['capienza_oraria'][l]>0:
                            t['capienza_oraria'][l]=t['capienza_oraria'][l]-1
                            activity=ET.SubElement(plan,'activity')
                            activity.attrib['type']='turism'
                            activity.attrib['facility']=str(t['id'])
                            coord=trans_coord(t['coord_X'],t['coord_Y'])
                            activity.attrib['x']=str(coord[0])
                            activity.attrib['y']=str(coord[1])
                            activity.attrib['end_time']=routines[a['id']][l]
                            leg=ET.SubElement(plan,'leg')
                            leg.attrib['mode']=mov
                            return True
                else:
                    for tl in tempolibero:
                        if tl['capienza_oraria'][l]>0:
                            tl['capienza_oraria'][l]=tl['capienza_oraria'][l]-1
                            activity=ET.SubElement(plan,'activity')
                            activity.attrib['type']='leisure'
                            activity.attrib['facility']=str(tl['id'])
                            coord=trans_coord(tl['coord_X'],tl['coord_Y'])
                            activity.attrib['x']=str(coord[0])
                            activity.attrib['y']=str(coord[1])
                            activity.attrib['end_time']=routines[a['id']][l]
                            leg=ET.SubElement(plan,'leg')
                            leg.attrib['mode']=mov
                            return True
            return False
        
        continua=False
        while not continua:
            continua=random.choice([casa,serata])()
        
    if l==12: #Rientro a casa
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='home'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])

In [30]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

for a in agenti_non_attivi:

    person = ET.SubElement(root, 'person')
    person.attrib['id'] = str(a['id'])
    
    if a['automunito']:
        mov='car'
    else:
        mov=random.choice(['walk','bike'])

    plan=ET.SubElement(person,'plan')
    
    routine=random.choice(routine_non_attivi);
    
    for l in routine:
        assegna_attivita_non_attivi(l,plan,a,mov)
        
print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-04-09 14:23:34
Fine = 2021-04-09 14:50:13


_________________

In [31]:
#Salvo population.xml
with open('population.xml', 'wb') as f:
    f.write('<?xml version="1.0"?>\n<!DOCTYPE population SYSTEM "http://www.matsim.org/files/dtd/population_v6.dtd">'.encode('utf8'))
    ET.ElementTree(root).write(f, 'utf-8')

________________